In general this was meant to originally convey the effects of the visionscore on several factors but as you can see at the end I mostly analysed the effect of a gold lead at minute 15 on the impact of the result for the relative sides and also the how much that increases win probability we can see a causal effect of 0.25 for Blue side and 0.27 for red side given that we have a gold lead of minute 15 

In [74]:
import pandas as pd 
import numpy as np 

In [75]:
#import files 
game_metadata = pd.read_csv('./out/game_metadata.csv')
game_player_stats = pd.read_csv('./out/game_player_stats.csv')
matches_df_original = pd.read_csv('./out/matches_df.csv')
matches_df = matches_df_original.copy()
game_player_stats = game_player_stats[game_player_stats['visionscore'] != 0.0]
zero_vs_count = (game_player_stats['visionscore'] == 0.0).sum()
print(game_player_stats.dtypes)

gameid                object
playerid              object
teamid                object
side                  object
position              object
champion              object
result                 int64
kills                  int64
deaths                 int64
assists                int64
damagetochampions    float64
visionscore          float64
earnedgold           float64
total cs             float64
golddiffat15         float64
csdiffat15           float64
xpdiffat15           float64
dtype: object


In [76]:
# Assign each team in a game to team_A or team_B
# We'll use rank within each game to determine side
game_player_stats = game_player_stats.copy()
game_player_stats['team_side'] = (
    game_player_stats.groupby('gameid')['teamid']
    .transform(lambda x: pd.factorize(x)[0])  # 0 and 1
)

# Map 0 -> team_A, 1 -> team_B
team_map = {0: 'team_A', 1: 'team_B'}
game_player_stats['team_side'] = game_player_stats['team_side'].map(team_map)

# Aggregate vision score & kills per team side
team_stats = (
    game_player_stats
    .groupby(['gameid', 'team_side'], as_index=False)
    .agg(
        vision_score=('visionscore', 'sum'),
        team_kills=('kills', 'sum')
    )
)

# Pivot to wide format with just team_A / team_B columns
team_stats_wide = team_stats.pivot(index='gameid', columns='team_side')
team_stats_wide.columns = [f"{stat}_{side}" for stat, side in team_stats_wide.columns]
team_stats_wide = team_stats_wide.reset_index()

# Merge into matches_df
matches_df = matches_df.merge(team_stats_wide, on='gameid', how='left')


In [77]:
game_player_stats['abs_golddiff_15'] = game_player_stats['golddiffat15'].abs()


# Take one row per game (any row is fine, since it's the same for all players in the game)
golddiff_per_game_abs = game_player_stats.drop_duplicates(subset=['gameid'])[['gameid', 'abs_golddiff_15']]
golddiff_per_game =game_player_stats.drop_duplicates(subset=['gameid'])[['gameid', 'golddiffat15']]
# Merge into matches_df
matches_df = matches_df.merge(golddiff_per_game_abs, on='gameid', how='left')
matches_df = matches_df.merge(golddiff_per_game, on='gameid', how='left')

In [78]:
matches_df = matches_df[~matches_df['vision_score_team_A'].isnull()]
matches_df = matches_df[~matches_df['vision_score_team_B'].isnull()]
matches_df = matches_df[~matches_df['team_kills_team_A'].isnull()]
matches_df = matches_df[~matches_df['team_kills_team_B'].isnull()]

In [79]:
print(matches_df.dtypes)
matches_df['total_kills'] = matches_df['team_kills_team_A'] + matches_df['team_kills_team_B']

Unnamed: 0               int64
gameid                  object
date                    object
league                  object
playoffs                 int64
patch                  float64
gamelength               int64
days_since_patch         int64
team_a                  object
team_b                  object
winner                  object
recent_patch             int64
elo_a                  float64
elo_b                  float64
winstreak_a            float64
winstreak_b            float64
elo_diff               float64
underdog                object
underdog_side_adv        int64
upset                    int64
vision_score_team_A    float64
vision_score_team_B    float64
team_kills_team_A      float64
team_kills_team_B      float64
abs_golddiff_15        float64
golddiffat15           float64
dtype: object


In [80]:
from dowhy import CausalModel

# Define DAG
causal_graph = """
digraph {
    recent_patch -> total_kills
    elo_diff -> total_kills
    vision_score_team_A -> total_kills
    vision_score_team_B -> total_kills
}
"""

# Build causal model directly on matches_df
model = CausalModel(
    data=matches_df,
    treatment="recent_patch",
    outcome="total_kills",
    graph=causal_graph
)

# Identify effect
identified_estimand = model.identify_effect()

# Estimate causal effect using linear regression (backdoor)
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print("Causal Estimate:", estimate.value)
#print(estimate)

Causal Estimate: 1.064035882396965


In [81]:
print(matches_df.dtypes)

Unnamed: 0               int64
gameid                  object
date                    object
league                  object
playoffs                 int64
patch                  float64
gamelength               int64
days_since_patch         int64
team_a                  object
team_b                  object
winner                  object
recent_patch             int64
elo_a                  float64
elo_b                  float64
winstreak_a            float64
winstreak_b            float64
elo_diff               float64
underdog                object
underdog_side_adv        int64
upset                    int64
vision_score_team_A    float64
vision_score_team_B    float64
team_kills_team_A      float64
team_kills_team_B      float64
abs_golddiff_15        float64
golddiffat15           float64
total_kills            float64
dtype: object


In [82]:
from dowhy import CausalModel

# Define DAG
causal_graph = """
digraph {
    elo_diff -> abs_golddiff_15
}
"""

# Build causal model directly on matches_df
model = CausalModel(
    data=matches_df,
    treatment="elo_diff",
    outcome="abs_golddiff_15",
    graph=causal_graph
)

# Identify effect
identified_estimand = model.identify_effect()

# Estimate causal effect using linear regression (backdoor)
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print("Causal Estimate:", estimate.value)
#print(estimate)

Causal Estimate: -0.11596670763367456


In [83]:
from dowhy import CausalModel

# Define DAG
causal_graph = """
digraph {
    vision_score_team_A -> golddiffat15
}
"""

# Build causal model directly on matches_df
model = CausalModel(
    data=matches_df,
    treatment="vision_score_team_A",
    outcome="golddiffat15",
    graph=causal_graph
)

# Identify effect
identified_estimand = model.identify_effect()

# Estimate causal effect using linear regression (backdoor)
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print("Causal Estimate:", estimate.value)
#print(estimate)

Causal Estimate: 0.8663837708127176


In [84]:
from dowhy import CausalModel

matches_df['vision_diff'] = matches_df['vision_score_team_A']-matches_df['vision_score_team_B']
# Define DAG
causal_graph = """
digraph {
    elo_diff -> vision_diff
}
"""

# Build causal model directly on matches_df
model = CausalModel(
    data=matches_df,
    treatment="elo_diff",
    outcome="vision_diff",
    graph=causal_graph
)

# Identify effect
identified_estimand = model.identify_effect()

# Estimate causal effect using linear regression (backdoor)
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print("Causal Estimate:", estimate.value)
#print(estimate)

Causal Estimate: 0.0015355752159327274


Interesting would be to look at how large of a gold lead it takes to have a causal effect on outcome 

In [85]:
print(matches_df.dtypes)

Unnamed: 0               int64
gameid                  object
date                    object
league                  object
playoffs                 int64
patch                  float64
gamelength               int64
days_since_patch         int64
team_a                  object
team_b                  object
winner                  object
recent_patch             int64
elo_a                  float64
elo_b                  float64
winstreak_a            float64
winstreak_b            float64
elo_diff               float64
underdog                object
underdog_side_adv        int64
upset                    int64
vision_score_team_A    float64
vision_score_team_B    float64
team_kills_team_A      float64
team_kills_team_B      float64
abs_golddiff_15        float64
golddiffat15           float64
total_kills            float64
vision_diff            float64
dtype: object


In [86]:
matches_df['large_lead_A'] = (matches_df['golddiffat15'] > 1000).astype(int)
matches_df['large_lead_B'] = (matches_df['golddiffat15'] < -1000).astype(int)
print(matches_df.dtypes)

Unnamed: 0               int64
gameid                  object
date                    object
league                  object
playoffs                 int64
patch                  float64
gamelength               int64
days_since_patch         int64
team_a                  object
team_b                  object
winner                  object
recent_patch             int64
elo_a                  float64
elo_b                  float64
winstreak_a            float64
winstreak_b            float64
elo_diff               float64
underdog                object
underdog_side_adv        int64
upset                    int64
vision_score_team_A    float64
vision_score_team_B    float64
team_kills_team_A      float64
team_kills_team_B      float64
abs_golddiff_15        float64
golddiffat15           float64
total_kills            float64
vision_diff            float64
large_lead_A             int64
large_lead_B             int64
dtype: object


In [87]:
matches_df['resultA'] = (matches_df['winner'] == 'team_a').astype(int)

from dowhy import CausalModel

# Define DAG
causal_graph = """
digraph {
    large_lead_A -> resultA
}
"""

# Build causal model directly on matches_df
model = CausalModel(
    data=matches_df,
    treatment="large_lead_A",
    outcome="resultA",
    graph=causal_graph
)

# Identify effect
identified_estimand = model.identify_effect()

# Estimate causal effect using linear regression (backdoor)
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print("Causal Estimate:", estimate.value)
#print(estimate)

Causal Estimate: 0.2530807371317741


This describes what happens if red side has a large gold lead and above is when blue side has 

In [88]:
matches_df['resultB'] = (matches_df['winner'] != 'team_a').astype(int)

from dowhy import CausalModel

# Define DAG
causal_graph = """
digraph {
    large_lead_B -> resultB
}
"""

# Build causal model directly on matches_df
model = CausalModel(
    data=matches_df,
    treatment="large_lead_B",
    outcome="resultB",
    graph=causal_graph
)

# Identify effect
identified_estimand = model.identify_effect()

# Estimate causal effect using linear regression (backdoor)
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print("Causal Estimate:", estimate.value)
#print(estimate)

Causal Estimate: 0.2775275750736432


Basic function that implements the expected outcome given the elos of team A and B so that we can look at the winrate of blue and red ahead as well as the winrate before the match 

In [89]:
def elo_win_prob(rating_A, rating_B):
    return 1 / (1 + 10 ** ((rating_B - rating_A) / 400))

In [90]:
#We now include it into the dataframe
matches_df['win_prob_A'] = elo_win_prob(matches_df['elo_a'], matches_df['elo_b'])
matches_df['win_prob_B'] = 1 - matches_df['win_prob_A']


In [91]:
#Use this instead of the underdog column to see whether or not we have an increase 
#Could later do this but instead of doing two teams per match we map how good in general 
#win probability increases for underdogs but since outcome are realitve close 
#We can assume the mean of the two to be the outcome in general 
matches_df['underdog_A'] = matches_df['win_prob_A'] < 0.5
matches_df['underdog_B'] = matches_df['win_prob_B'] < 0.5

# Mask for underdogs with a large lead
underdog_A_mask = (matches_df['underdog_A']) & (matches_df['large_lead_A'] == 1)
underdog_B_mask = (matches_df['underdog_B']) & (matches_df['large_lead_B'] == 1)

# Compute increase for each side
increase_underdog_A = matches_df.loc[underdog_A_mask, 'resultA'].mean() - matches_df.loc[underdog_A_mask, 'win_prob_A'].mean()
increase_underdog_B = matches_df.loc[underdog_B_mask, 'resultB'].mean() - matches_df.loc[underdog_B_mask, 'win_prob_B'].mean()

# Aggregate both sides
increase_underdog_avg = (increase_underdog_A + increase_underdog_B) / 2

print(f"Team A underdog: increase = {increase_underdog_A:.3f}")
print(f"Team B underdog: increase = {increase_underdog_B:.3f}")
print(f"Average underdog increase = {increase_underdog_avg:.3f}")

Team A underdog: increase = 0.302
Team B underdog: increase = 0.279
Average underdog increase = 0.291


In [92]:
blue_mask = matches_df['large_lead_A'] == 1
red_mask = matches_df['large_lead_B'] == 1

# Compute win rates
blue_ahead_winrate = matches_df.loc[blue_mask, 'resultA'].mean()
red_ahead_winrate = matches_df.loc[red_mask, 'resultB'].mean()

print(f"Blue ahead winrate: {blue_ahead_winrate:.3f}")
print(f"Red ahead winrate: {red_ahead_winrate:.3f}")

Blue ahead winrate: 0.740
Red ahead winrate: 0.707


Both sides have a very large winrate for a goldlead of 1000 at minute 15

We find that in general we a large win probability given that we have a gold lead at minute 15 on both sides 
further we see that the causal effect of a large gold lead at minute 15 also increase the result_A by 0.25 and result_B by 0.27 

Now we want to analyse whether or not and how large the effect is on the win probability of each team 

In [93]:
# Team A: average win probability increase if ahead
win_prob_increase_A = matches_df.loc[matches_df['large_lead_A'] == 1, 'resultA'].mean() - matches_df['win_prob_A'].mean()

# Team B: average win probability increase if ahead
win_prob_increase_B = matches_df.loc[matches_df['large_lead_B'] == 1, 'resultB'].mean() - matches_df['win_prob_B'].mean()

print(f"Team A: avg increase in win probability with large lead = {win_prob_increase_A:.3f}")
print(f"Team B: avg increase in win probability with large lead = {win_prob_increase_B:.3f}")

Team A: avg increase in win probability with large lead = 0.222
Team B: avg increase in win probability with large lead = 0.224


On average our win probability for both sides win prob increases by 0.22 given a gold lead of 1000 at minute 15 and if we look at the underdogs we get an increase of about 0.29

In [94]:
print(matches_df.dtypes)
matches_df.to_csv('./out/matches_df.csv')

Unnamed: 0               int64
gameid                  object
date                    object
league                  object
playoffs                 int64
patch                  float64
gamelength               int64
days_since_patch         int64
team_a                  object
team_b                  object
winner                  object
recent_patch             int64
elo_a                  float64
elo_b                  float64
winstreak_a            float64
winstreak_b            float64
elo_diff               float64
underdog                object
underdog_side_adv        int64
upset                    int64
vision_score_team_A    float64
vision_score_team_B    float64
team_kills_team_A      float64
team_kills_team_B      float64
abs_golddiff_15        float64
golddiffat15           float64
total_kills            float64
vision_diff            float64
large_lead_A             int64
large_lead_B             int64
resultA                  int64
resultB                  int64
win_prob

We now want to visualise the causal effect of gold lead on each sides result 

In [95]:
print(matches_df.dtypes)

Unnamed: 0               int64
gameid                  object
date                    object
league                  object
playoffs                 int64
patch                  float64
gamelength               int64
days_since_patch         int64
team_a                  object
team_b                  object
winner                  object
recent_patch             int64
elo_a                  float64
elo_b                  float64
winstreak_a            float64
winstreak_b            float64
elo_diff               float64
underdog                object
underdog_side_adv        int64
upset                    int64
vision_score_team_A    float64
vision_score_team_B    float64
team_kills_team_A      float64
team_kills_team_B      float64
abs_golddiff_15        float64
golddiffat15           float64
total_kills            float64
vision_diff            float64
large_lead_A             int64
large_lead_B             int64
resultA                  int64
resultB                  int64
win_prob

In [97]:
bins = np.arange(-4000, 4001, 500)  
matches_df['gold_bin'] = pd.cut(matches_df['golddiffat15'], bins=bins)

In [96]:
def causal_effect_per_bin(df, treatment_col, outcome_col, confounders=['elo_diff']):
    effects = []
    for b in df['gold_bin'].cat.categories:
        subset = df[df['gold_bin'] == b]
        if subset[treatment_col].nunique() < 2:
            continue  # skip bins without variation in treatment
        model = CausalModel(
            data=subset,
            treatment=treatment_col,
            outcome=outcome_col,
            common_causes=confounders
        )
        identified_estimand = model.identify_effect()
        estimate = model.estimate_effect(identified_estimand, method_name="backdoor.linear_regression")
        effects.append((b.mid, estimate.value))  # use mid-point of bin
    return pd.DataFrame(effects, columns=['gold_mid', 'effect'])

In [98]:
#Calculate the causal effect for A and for B for each bin 
effects_A = causal_effect_per_bin(matches_df, 'large_lead_A', 'resultA')
effects_B = causal_effect_per_bin(matches_df, 'large_lead_B', 'resultB')

In [ ]:
#Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(effects_A['gold_mid'], effects_A['effect'], marker='o', label='Blue Side (Team A)')
plt.plot(effects_B['gold_mid'], effects_B['effect'], marker='s', label='Red Side (Team B)')
plt.axhline(0, color='gray', linestyle='--')
plt.xlabel('Gold Lead at 15 min')
plt.ylabel('Increase in Win Probability')
plt.title('Causal Effect of Gold Lead on Win Probability')
plt.legend()
plt.grid(True)
plt.show()